## Bài 1 

In [60]:
import pandas as pd
import sqlite3

df = pd.DataFrame({
    'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4'],
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6]
})

conn = sqlite3.connect(":memory:")
df.to_sql("data", conn, index=False)

# 
query = """
SELECT
    (n * sum_ab - sum_a * sum_b) /
    (SQRT(n * sum_a2 - sum_a * sum_a) * SQRT(n * sum_b2 - sum_b * sum_b)) AS r_ab
FROM (
    SELECT
        COUNT(*) AS n,
        SUM(A) AS sum_a,
        SUM(B) AS sum_b,
        SUM(A * B) AS sum_ab,
        SUM(A * A) AS sum_a2,
        SUM(B * B) AS sum_b2
    FROM data
) AS stats;
"""

result_sqlite = pd.read_sql(query, conn)
print("Kết quả :")
print(result_sqlite)


Kết quả :
       r_ab
0  0.691051


## Bài 2


In [41]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute('''
CREATE TABLE mau_xe (
    day INTEGER,
    model_A FLOAT,
    model_B FLOAT,
    model_C FLOAT
)
''')

# Nhập dữ liệu vào bảng
data = [
    (1, 8, 9, 7),
    (2, 7.5, 8.5, 7),
    (3, 6, 7, 8),
    (4, 7, 6, 5)
]
cursor.executemany('INSERT INTO mau_xe (day, model_A, model_B, model_C) VALUES (?, ?, ?, ?)', data)
conn.commit()

# Truy vấn dữ liệu từ bảng và chuyển thành DataFrame
df = pd.read_sql_query('SELECT * FROM mau_xe', conn)
print(df)

# Chuyển dữ liệu sang dạng quan hệ (long format)
df_lg = pd.melt(df, id_vars=['day'], var_name='model', value_name='score')

# In dữ liệu sau khi chuyển
print(" Dữ liệu dạng quan hệ:")
print(df_lg)

# Làm tròn điểm số để có dữ liệu rời rạc
df_lg['rounded_score'] = df_lg['score'].round()

# Tạo bảng tần suất giữa mẫu xe và điểm số
bang_tan_suat = pd.crosstab(df_lg['model'], df_lg['rounded_score'])

# In bảng tần suất
print(" Bảng tần suất:")
print(bang_tan_suat)

# Thực hiện kiểm định χ2
chi2, p, dof, expected = chi2_contingency(bang_tan_suat)

# In kết quả
print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
print(f"Tần suất dự kiến: \n{expected}")

# Đánh giá kết quả kiểm định
if p < 0.05:
    print(" => Có sự khác biệt đáng kể giữa các mẫu xe (bác bỏ H0).")
else:
    print(" => Không có sự khác biệt đáng kể giữa các mẫu xe (chấp nhận H0).")


   day  model_A  model_B  model_C
0    1      8.0      9.0      7.0
1    2      7.5      8.5      7.0
2    3      6.0      7.0      8.0
3    4      7.0      6.0      5.0
 Dữ liệu dạng quan hệ:
    day    model  score
0     1  model_A    8.0
1     2  model_A    7.5
2     3  model_A    6.0
3     4  model_A    7.0
4     1  model_B    9.0
5     2  model_B    8.5
6     3  model_B    7.0
7     4  model_B    6.0
8     1  model_C    7.0
9     2  model_C    7.0
10    3  model_C    8.0
11    4  model_C    5.0
 Bảng tần suất:
rounded_score  5.0  6.0  7.0  8.0  9.0
model                                 
model_A          0    1    1    2    0
model_B          0    1    1    1    1
model_C          1    0    2    1    0
Chi-squared: 6.0
P-value: 0.6472318887822313
Degrees of freedom: 8
Tần suất dự kiến: 
[[0.33333333 0.66666667 1.33333333 1.33333333 0.33333333]
 [0.33333333 0.66666667 1.33333333 1.33333333 0.33333333]
 [0.33333333 0.66666667 1.33333333 1.33333333 0.33333333]]
 => Không có sự khác bi

## Bài 3

In [49]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng 
cursor.execute('''
CREATE TABLE flights (
    departure_time INTEGER
)
''')

# Nhập dữ liệu vào bảng (thời gian dưới dạng số nguyên)
data = [
    (830,),
    (1445,),
    (144,),
    (24,),
    (0,),
    (1230,),
    (19,),
    (2359,)
]

cursor.executemany('INSERT INTO flights (departure_time) VALUES (?)', data)
conn.commit()

# Truy vấn dữ liệu từ bảng flights
df = pd.read_sql_query('SELECT * FROM flights', conn)
print("Dữ liệu ban đầu:")
print(df)

# Hàm chuyển đổi số nguyên thành định dạng thời gian 
def convert_to_time_format(departure_time):
    hour = departure_time // 100
    minute = departure_time % 100
    time_str = f"{hour:02}:{minute:02}"
    return pd.to_datetime(time_str, format="%H:%M").strftime('%I:%M %p')

# Áp dụng hàm chuyển đổi cho cột departure_time
df['departure_time_converted'] = df['departure_time'].apply(convert_to_time_format)

# In kết quả
print(" Dữ liệu sau khi chuyển đổi:")
print(df)


Dữ liệu ban đầu:
   departure_time
0             830
1            1445
2             144
3              24
4               0
5            1230
6              19
7            2359
 Dữ liệu sau khi chuyển đổi:
   departure_time departure_time_converted
0             830                 08:30 AM
1            1445                 02:45 PM
2             144                 01:44 AM
3              24                 12:24 AM
4               0                 12:00 AM
5            1230                 12:30 PM
6              19                 12:19 AM
7            2359                 11:59 PM


## Bài 4


In [61]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(':memory:')  

# Tạo bảng và chèn dữ liệu mẫu
conn.execute('''
CREATE TABLE students_scores (
    student_id INTEGER,
    name TEXT,
    score FLOAT
);
''')

# Chèn dữ liệu mẫu vào bảng 'students_scores'
data = [
    (1, 'An', 8.5),
    (2, 'Bình', 9.2),
    (3, 'Chi', 7.4),
    (4, 'Dũng', 9.9),
    (5, 'Hằng', 8.8),
    (6, 'Hoàng', 9.5),
    (7, 'Lan', 10.0),
    (8, 'Mai', 7.0),
    (9, 'Nam', 6.0),
    (10, 'Oanh', 9.2),
    (11, 'Phúc', 8.5),
    (12, 'Quang', 9.8)
]
conn.executemany('INSERT INTO students_scores (student_id, name, score) VALUES (?, ?, ?);', data)
conn.commit()

# Truy vấn SQL để tìm ngoại lệ bằng MAD
query = '''
WITH stats AS (
    SELECT 
        score
    FROM students_scores
    ORDER BY score
    LIMIT 1 OFFSET (SELECT COUNT(*)/2 FROM students_scores)
),
mad AS (
    SELECT
        AVG(ABS(score - (SELECT score FROM stats))) AS mad_value
    FROM students_scores
),
outliers AS (
    SELECT
        student_id,
        name,
        score,
        ABS(score - (SELECT score FROM stats)) AS deviation
    FROM students_scores
)
SELECT 
    student_id,
    name,
    score,
    deviation,
    CASE
        WHEN deviation > 1.5 * (SELECT mad_value FROM mad) THEN 'Outlier'
        ELSE 'Not Outlier'
    END AS is_outlier
FROM outliers;
'''

# Thực thi câu lệnh SQL và lấy kết quả vào pandas DataFrame
outliers_df = pd.read_sql_query(query, conn)

# Hiển thị kết quả
outliers_df


,student_id,name,score,deviation,is_outlier
0,1,An,8.5,0.7,Not Outlier
1,2,Bình,9.2,0.0,Not Outlier
2,3,Chi,7.4,1.8,Outlier
3,4,Dũng,9.9,0.7,Not Outlier
4,5,Hằng,8.8,0.4,Not Outlier
5,6,Hoàng,9.5,0.3,Not Outlier
6,7,Lan,10.0,0.8,Not Outlier
7,8,Mai,7.0,2.2,Outlier
8,9,Nam,6.0,3.2,Outlier
9,10,Oanh,9.2,0.0,Not Outlier


## Bài 5 

In [62]:
import sqlite3
import pandas as pd


conn = sqlite3.connect(':memory:')  

# Tạo bảng Patient 
conn.execute('''
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight INTEGER,
    height INTEGER
)
''')

# Chèn dữ liệu mẫu mới vào bảng Patient
conn.execute('''
INSERT INTO Patient (last_name, weight, height) VALUES
('Nguyen', 70, 175),
('Tran', 65, 160),
('Pham', 60, 170),
('Nguyen', 70, 175),
('Le', 55, 160)
''')

# Thực thi câu lệnh SQL để tìm các cặp bệnh nhân có last_name và weight giống nhau
query = """
SELECT 
    p1.id AS patient_1, 
    p2.id AS patient_2,
    p1.last_name AS last_name_1, 
    p2.last_name AS last_name_2,
    p1.weight AS weight_1,
    p2.weight AS weight_2,
    CASE
        WHEN p1.last_name = p2.last_name AND p1.weight = p2.weight THEN 'cùng 1 người'
        WHEN p1.last_name = p2.last_name OR p1.weight = p2.weight THEN 'Có thể là cùng một người'
        ELSE 'Different person'
    END AS result
FROM Patient p1
JOIN Patient p2 ON (p1.last_name = p2.last_name OR p1.weight = p2.weight)
WHERE p1.id != p2.id;
"""

# Thực thi câu lệnh SQL và lấy kết quả
df = pd.read_sql(query, conn)

# In kết quả
print("Các cặp bệnh nhân được cho là cùng một người (dựa trên last_name và weight):")
print(df)


Các cặp bệnh nhân được cho là cùng một người (dựa trên last_name và weight):
   patient_1  patient_2 last_name_1 last_name_2  weight_1  weight_2  \
0          1          4      Nguyen      Nguyen        70        70   
1          4          1      Nguyen      Nguyen        70        70   

         result  
0  cùng 1 người  
1  cùng 1 người  
